In [2]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Import training data to pandas dataframe
df = pd.read_csv("train.csv")
# Trimming Down
df_bm = df.loc[:,['Neighborhood','GrLivArea','YearBuilt','OverallQual','CentralAir','SalePrice','FullBath','HalfBath','BsmtFullBath','BsmtHalfBath','TotalBsmtSF','BsmtUnfSF']]


In [3]:
# Creating total bathroom count TotBathroom
TotBathroom = df_bm.FullBath + 0.5 * df_bm.HalfBath \
+ df_bm.BsmtFullBath + (0.5) * df_bm.BsmtHalfBath

df_bm['TotBathroom'] = TotBathroom

# Remove the one's we don't need anymore
del df_bm['FullBath']
del df_bm['HalfBath']
del df_bm['BsmtFullBath']
del df_bm['BsmtHalfBath']

In [4]:
# Creating total square foot TotSF
TotSF = df_bm.GrLivArea + df_bm.TotalBsmtSF - df_bm.BsmtUnfSF
# Adding to dataset
df_bm['TotSF'] = TotSF

# Remove the ones we don't need anymore
del df_bm['GrLivArea']
del df_bm['TotalBsmtSF']
del df_bm['BsmtUnfSF']


In [5]:
# Data Preprocessing

# deal with missing values in textual columns 
#replace missing neighborhood with a dummy variable 
df_bm['Neighborhood'].fillna('fill')
#Replace missing ac classification with Y since more likely
#and allows us to maintain binary classification
df_bm['CentralAir'].fillna('Y')

df_bm[:5]

,Neighborhood,YearBuilt,OverallQual,CentralAir,SalePrice,TotBathroom,TotSF
0,CollgCr,2003,7,Y,208500,3.5,2416
1,Veenker,1976,6,Y,181500,2.5,2240
2,CollgCr,2001,7,Y,223500,3.5,2272
3,Crawfor,1915,7,Y,140000,2.0,1933
4,NoRidge,2000,8,Y,250000,3.5,2853


In [6]:
df_bm.columns

Index([u'Neighborhood', u'YearBuilt', u'OverallQual', u'CentralAir',
       u'SalePrice', u'TotBathroom', u'TotSF'],
      dtype='object')

# Basic Linear Regression

# Get input data


In [9]:
df_X = df_bm.loc[:,['YearBuilt', 'TotSF', 'TotBathroom']]
df_y = df_bm.loc[:, ['SalePrice']]

# Data Preprocessing

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_X[['YearBuilt', 'TotSF', 'TotBathroom']] \
    = scaler.fit_transform(df_X[['YearBuilt', 'TotSF', 'TotBathroom']])

#view transformed set
df_X[:5]

,YearBuilt,TotSF,TotBathroom
0,1.050994,0.529516,1.642256
1,0.156734,0.302404,0.368581
2,0.984752,0.343697,1.642256
3,-1.863632,-0.093752,-0.268257
4,0.951632,1.093426,1.642256


In [49]:
import torch
from torch.autograd import Variable

In [50]:
X = Variable(torch.Tensor(df_X.values.tolist()))
y = Variable(torch.Tensor(df_y.values.tolist()))

Variable containing:
 2.0850e+05
 1.8150e+05
 2.2350e+05
     ⋮      
 2.6650e+05
 1.4212e+05
 1.4750e+05
[torch.FloatTensor of size 1460x1]

# Linear Regression

In [51]:
W = Variable(torch.ones(3, 1), requires_grad=True)
b = Variable(torch.zeros(1460, 1), requires_grad=True)

In [52]:
import torch.nn as nn

learning_rate = 1
optimizer = torch.optim.Adam([W, b], lr=learning_rate)
loss_fxn = nn.MSELoss()
t = 0
stillTraining = True
while (stillTraining):
    predictions = X.mm(W) + b
    losses = loss_fxn(predictions, y)
    if t % 10000 == 0:
        print losses.data[0]
    if (losses.data[0] < 1):
        print losses.data[0]
        stillTraining = False
    t = t + 1 
    optimizer.zero_grad()   
    losses.backward()
    optimizer.step()

39038943232.0
33250772992.0
28784572416.0
25366571008.0
22319093760.0
19552708608.0
17028392960.0
14703465472.0
12575782912.0
10644757504.0
8909703168.0
7368880640.0
6019908096.0
4858821632.0
3878746112.0
3067345408.0
2409041664.0
1884491264.0
1473253632.0
1153851648.0
906195328.0
713968640.0
564774656.0
449220736.0
359978816.0
290722080.0
236055824.0
191772416.0
155622656.0
126096360.0
102035144.0
82213608.0
65710312.0
51764076.0
40011904.0
30230606.0
22280650.0
15946502.0
10978833.0
7177645.0
4412192.5
2484108.0
1148490.375
375151.125
58816.953125
0.996343910694


# Testing